In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install findspark

In [ ]:
!pip install pyspark

In [ ]:
import findspark
import numpy as np

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pysparkML").getOrCreate()

In [ ]:
train = spark.read.csv('../input/car-acceptability-prediction/train.csv', inferSchema=True, header=True)


In [ ]:
test = spark.read.csv('../input/car-acceptability-prediction/test.csv', inferSchema=True, header=True)


In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler,IndexToString

from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator


In [ ]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="acceptability", outputCol="label_index").fit(train)
train = labelIndexer.transform(train)

## transform other features into indexes

buying_Indexer = StringIndexer(inputCol="buying_price", outputCol="buying_index")
maint_Indexer = StringIndexer(inputCol="maintenance_price", outputCol="maint_index")
doors_Indexer = StringIndexer(inputCol="number_of_doors", outputCol="doors_index")
carry_Indexer = StringIndexer(inputCol="carry_capacity", outputCol="carry_index")
trunk_size_Indexer = StringIndexer(inputCol="trunk_size", outputCol="trunk_size_index")
safty_Indexer = StringIndexer(inputCol="safety", outputCol="safety_index")

assembler = VectorAssembler( inputCols=["buying_index", "maint_index", "doors_index","carry_index", "trunk_size_index", "safety_index"], outputCol="features_index")




In [ ]:
from pyspark.ml.classification import RandomForestClassifier

#model_DecisionTreeClassifier = DecisionTreeClassifier(labelCol= "label_index", featuresCol="features_index")
RandomForest_model = RandomForestClassifier(labelCol = "label_index", featuresCol="features_index" ,  numTrees =256)
    # Chain featurizers in a Pipeline
pipeline = Pipeline(stages=[ buying_Indexer ,maint_Indexer, doors_Indexer, carry_Indexer, trunk_size_Indexer, safty_Indexer, 
                            assembler,RandomForest_model])

model = pipeline.fit(train)


In [ ]:

#make predictions
predictions = model.transform(test)

In [ ]:
#display
#predictions.select("car_id","buying_price","maintenance_price","number_of_doors","carry_capacity","trunk_size","safety","prediction").show(10)
predicted = predictions.select("car_id","prediction")



In [ ]:
predicted.show(20)

acc =1 , unacc =0 good =2 vgood =3

In [ ]:
from pyspark.sql.functions import regexp_replace
data = predicted.withColumn('prediction', regexp_replace('prediction', '1.0', 'acc')) \
         .withColumn('prediction', regexp_replace('prediction', '0.0', 'unacc')) \
         .withColumn('prediction', regexp_replace('prediction', '2.0', 'good')) \
         .withColumn('prediction', regexp_replace('prediction', '3.0', 'vgood')) \
         .withColumnRenamed("prediction", "acceptability" ) \



In [ ]:
data.show(20)

In [ ]:
data.toPandas().to_csv('c.csv',index=False)

